# 爱科研 - 利用神经网络预测学生录取情况

在该 notebook 中，我们基于以下三条数据预测了加州大学洛杉矶分校 (UCLA) 的研究生录取情况：

- GRE 分数（测试）即 GRE Scores (Test)

- GPA 分数（成绩）即 GPA Scores (Grades)

- 评级（1-4）即 Class rank (1-4)

数据集来源：http://www.ats.ucla.edu/

## 加载数据

为了加载数据并很好地进行格式化，我们将使用两个非常有用的包，即 Pandas 和 Numpy。 你可以在这里此文档：

- https://pandas.pydata.org/pandas-docs/stable/
- https://docs.scipy.org/

In [ ]:
# Importing pandas and numpy
import pandas as pd
import numpy as np

# Reading the csv file into a pandas DataFrame
data = pd.read_csv('student_data.csv')

# Printing out the first 10 rows of our data
data[:10]

In [ ]:
# Importing matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Function to help us plot
def plot_points(data):
    X = np.array(data[["gre","gpa"]])
    y = np.array(data["admit"])
    admitted = X[np.argwhere(y==1)]
    rejected = X[np.argwhere(y==0)]
    plt.scatter([s[0][0] for s in rejected], [s[0][1] for s in rejected], s = 25, color = 'red', edgecolor = 'k')
    plt.scatter([s[0][0] for s in admitted], [s[0][1] for s in admitted], s = 25, color = 'cyan', edgecolor = 'k')
    plt.xlabel('Test (GRE)')
    plt.ylabel('Grades (GPA)')
    
# Plotting the points
plot_points(data)
plt.show()

粗略地说，它看起来像是，成绩 （grades) 和测试(test) 分数 高的学生通过了，而得分低的学生却没有，但数据并没有如我们所希望的那样，很好地分离。 也许将评级 (rank) 考虑进来会有帮助？ 接下来我们将绘制 4 个图，每个图代表一个级别。

In [ ]:
# Separating the ranks
data_rank1 = data[data["rank"]==1]
data_rank2 = data[data["rank"]==2]
data_rank3 = data[data["rank"]==3]
data_rank4 = data[data["rank"]==4]

# Plotting the graphs
plot_points(data_rank1)
plt.title("Rank 1")
plt.show()
plot_points(data_rank2)
plt.title("Rank 2")
plt.show()
plot_points(data_rank3)
plt.title("Rank 3")
plt.show()
plot_points(data_rank4)
plt.title("Rank 4")
plt.show()

现在看起来更棒啦，看上去评级越低，录取率越高。 让我们使用评级 (rank) 作为我们的输入之一。 为了做到这一点，我们应该对它进行一次one-hot 编码。

## 将评级进行 One-hot 编码
我们将在 pandas 中使用 `get_dummies` 函数。

In [ ]:
# Make dummy variables for rank
one_hot_data = pd.concat([data, pd.get_dummies(data['rank'], prefix='rank')], axis=1)

# Drop the previous rank column
one_hot_data = one_hot_data.drop('rank', axis=1)

# Print the first 10 rows of our data
one_hot_data[:10]

## 缩放数据
下一步是缩放数据。 我们注意到成绩 (grades) 的范围是 1.0-4.0，而测试分数 （test scores) 的范围大概是 200-800，这个范围要大得多。 这意味着我们的数据存在偏差，使得神经网络很难处理。 让我们将两个特征放在 0-1 的范围内，将分数除以 4.0，将测试分数除以 800。

In [ ]:
# Copying our data
processed_data = one_hot_data[:]

# Scaling the columns
processed_data['gre'] = processed_data['gre']/800
processed_data['gpa'] = processed_data['gpa']/4.0
processed_data[:10]

## 将数据分成训练集和验证集

为了测试我们的算法，我们将数据分为训练集和验证集。 测试集的大小将占总数据的 20％。

In [ ]:
sample = np.random.choice(processed_data.index, size=int(len(processed_data)*0.8), replace=False)
train_data, val_data = processed_data.iloc[sample], processed_data.drop(sample)

print("Number of training samples is", len(train_data))
print("Number of validation samples is", len(val_data))
print(train_data[:10])
print(val_data[:10])

## 将数据分成特征和目标（标签）
现在，在培训前的最后一步，我们将把数据分为特征 (features)（X）和目标 (targets)（y）。

另外，在 Keras 中，我们需要对输出进行 one-hot 编码。 我们将使用`to_categorical function` 来做到这一点。

In [ ]:
import keras

# Separate data and one-hot encode the output
# Note: We're also turning the data into numpy arrays, in order to train the model in Keras
features = np.array(train_data.drop('admit', axis=1))
targets = np.array(keras.utils.to_categorical(train_data['admit'], 2))
features_val = np.array(val_data.drop('admit', axis=1))
targets_val = np.array(keras.utils.to_categorical(val_data['admit'], 2))

print(features[:10])
print(targets[:10])

## 定义模型架构
我们将使用 Keras 来构建神经网络。

In [ ]:
# Imports
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

# Building the model
model = Sequential()
model.add(Dense(128, activation='sigmoid', input_shape=(6,)))
model.add(Dropout(.2))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## 训练模型

In [ ]:
# Training the model
history = model.fit(features, targets, validation_data=(features_val, targets_val),
                    epochs=200, batch_size=100, verbose=0)
print(history.history.keys())

In [ ]:
with mpl.rc_context(rc={'font.family': 'serif', 'font.size': 11}):
    fig = plt.figure(figsize=(9,3))
    ax1 = fig.add_subplot(121)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Crossentropy')
    ax1.set_title('Loss')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['train', 'val'], loc='upper right')
    #plt.ylim(0,450)
    ax2 = fig.add_subplot(122)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.legend(['train', 'val'], loc='lower right')
    plt.ylim(0.5,1)

## 模型评分

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(features, targets)
print("\n Training Accuracy:", score[1])
score = model.evaluate(features_val, targets_val)
print("\n Validation Accuracy:", score[1])

## 练习：玩转参数

你可以看到我们在训练过程中做了几个规定。 例如，对图层的数量，图层的大小，epoch 的数量等有所规定。
现在轮到你来玩转参数！ 你能提高准确性吗？ 以下是针对这些参数的其他建议。 我们将在课程后面学习如下概念的定义：

- 激活函数 (Activation function)：relu 和 sigmoid
- 损失函数 (Loss function)：categorical_crossentropy，mean_squared_error
- 优化器 (Optimizer)：rmsprop，adam，ada